In [1]:
%load_ext autoreload
%autoreload 2

from utils.all import *
log.basicConfig(level=log.INFO)
data = load_data('../data/test_set_inspection_rating_annotated', preprocess=True, adjudicated=True)
systems = set([x['system'] for x in data])
edit_types = set(data[0]['annotations'].keys())

ERROR:When processing sentence: vinayak - Batch 1, HIT 10 (ID 193). Caught error on: ''. Skipping...


In [7]:
len(systems)

7

In [6]:
len(set([x['original'] for x in data]))

20

In [2]:
import numpy as np
s = [len(s['original'].split(' ')) for s in data]
print(f"SimpEval Mean: {np.mean(s):.2f}, Std dev.: {np.std(s):.2f}", )

SimpEval Mean: 32.58, Std dev.: 10.46


In [3]:
with open('C:\\Users\\heine\\Documents\\research\\ts-annotation-tool\\data\\tmp\\asset\\dataset\\asset.test.orig', 'r') as file:
    lines = file.readlines()
    text = ' '.join(lines)
    asset = text.split('\n ')
s = [len(s.split(' ')) for s in asset]
print(f"ASSET Mean: {np.mean(s):.2f}, Std dev.: {np.std(s):.2f}", )

ASSET Mean: 19.72, Std dev.: 7.95


In [4]:
savefig = True

fig, ax_quality = plt.subplots(1, 3, figsize=(13, 3.5))
width = 0.7

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'TW Cen MT'

# plt.rcParams['font.family'] = 'serif'
# plt.rcParams['font.sans-serif'] = 'Times New Roman'

font_size_legend = 11
font_size_x_labels = 12 
font_size_y_label = 12
font_size_title = 15
legend_loc = (1, 0.5)

def fix_text(text, syntax_error=False):
    new_text = text.replace(' ', '\n').replace('t-', 't-\n')
    if syntax_error:
        new_text = f'Bad {new_text}'
    if 'level' in new_text:
        new_text = f'{new_text} Reorder'
        new_text = new_text.replace('Word-level ', 'Word-level\n')
    if 'Grammar' in new_text:
        new_text = new_text.replace('Grammar', 'Grammar\nError')
    return new_text

for plt_idx, family in enumerate(Family):
    out = get_edits_by_family(data, family, combine_humans=False)
    # Get the system labels by preserving the order of systems
    system_labels = [x for x in system_name_mapping if x in out.keys()]
    x = np.arange(len(system_labels))

    # Graph the quality edits
    quality_data = {system : out[system]['quality'] for system, _ in out.items()}
    bottom = [0 for x in range(len(system_labels))]
    if family == Family.CONTENT:
        quality_iterator = [Information.MORE, Information.LESS]
    elif family == Family.SYNTAX:
        quality_iterator = [x for x in ReorderLevel] + [Edit.STRUCTURE] + [Edit.SPLIT]
    elif family == Family.LEXICAL:
        quality_iterator = [Information.SAME]
    for quality_type in quality_iterator:
        val = [quality_data[label][quality_type] for label in system_labels]
        if sum(val) != 0:
            # Custom labels
            label = quality_type.value
            # if family == Family.SYNTAX:
            #     label = 'Quality ' + quality_type.value.lower() + ' edit'
            if quality_type == Information.SAME:
                label = 'Paraphrase'

            ax_quality[plt_idx].bar(x, val, width, bottom=bottom, label=label, color=color_mapping[quality_type])
        bottom = [bottom[i] + val[i] for i in range(len(val))]

    displayed_x_labels = [system_name_mapping[label] for label in system_labels]

    ax_quality[plt_idx].tick_params(labelsize=font_size_x_labels)
    ax_quality[plt_idx].set_xticks(np.arange(len(system_labels)))
    ax_quality[plt_idx].set_xticklabels(displayed_x_labels, rotation=30, ha="right")
    ax_quality[plt_idx].spines[['right', 'top']].set_visible(False)

    if family == Family.CONTENT:
        ax_quality[plt_idx].set_ylabel('# Quality Edits / Sent.', fontsize=font_size_y_label)

    handles, labels = ax_quality[plt_idx].get_legend_handles_labels()
    ax_quality[plt_idx].legend(handles[::-1], labels[::-1], loc='center left', bbox_to_anchor=legend_loc,
        fancybox=True, ncol=1, borderaxespad=1.,fontsize=font_size_legend,
        facecolor='white',edgecolor='black',framealpha=1,frameon=False,
        columnspacing=1,handlelength=1,handleheight=1,handletextpad=0.6,
        borderpad=0.2, alignment='left', 
        # title=f'{family.value} Quality', title_fontproperties={'weight': 500, 'size': font_size_legend, 'style': 'italic'}
        )

    for ha in ax_quality[plt_idx].legend_.legendHandles:
        ha.set_edgecolor("black")

    trans = mtrans.Affine2D().translate(10, 0)
    for t in ax_quality[plt_idx].get_xticklabels():
        t.set_transform(t.get_transform()+trans)

    # Set the margins a little higher than the max value
    max_quality = max([sum(x.values()) for x in quality_data.values()])
    tick_range_quality = np.arange(0, max_quality*1.2, step=0.5)
    ax_quality[plt_idx].set_yticks(tick_range_quality)

    for text in ax_quality[plt_idx].legend_.texts:
        text.set_text(fix_text(text.get_text()))

    ax_quality[0].set_title('Conceptual Quality', fontsize=font_size_title)
    ax_quality[1].set_title('Syntax Quality', fontsize=font_size_title)
    ax_quality[2].set_title('Lexical Quality', fontsize=font_size_title)

plt.tight_layout()
if savefig:
    out_filename = f'../paper/plot/quality-edits.pdf'
    plt.savefig(out_filename, format="pdf", bbox_inches='tight', pad_inches=0.0)
    plt.close(fig)
else:
    plt.show()

fig, ax_error = plt.subplots(1, 3, figsize=(13, 3.5))

for plt_idx, family in enumerate(Family):
    out = get_edits_by_family(data, family, combine_humans=False)
    # Get the system labels by preserving the order of systems
    system_labels = [x for x in system_name_mapping if x in out.keys()]
    x = np.arange(len(system_labels))

    # Graph the error edits
    error_data = {system : out[system]['error'] for system, _ in out.items()}
    bottom = [0 for x in range(len(system_labels))]
    if family == Family.CONTENT:
        error_iterator = [e for e in Error if e != Error.UNNECESSARY_INSERTION]
    elif family == Family.SYNTAX:
        error_iterator = [x for x in ReorderLevel] + [Edit.STRUCTURE] + [Edit.SPLIT]
    elif family == Family.LEXICAL:
        error_iterator = [Error.COMPLEX_WORDING, Quality.ERROR, Error.INFORMATION_REWRITE]
    for error_type in error_iterator:
        val = [error_data[label][error_type] for label in system_labels]

        # This is a really awful solution, but it would be too much to change the classification of this error
        if family == Family.LEXICAL and error_type == Error.UNNECESSARY_INSERTION:
            tmp = get_edits_by_family(data, Family.CONTENT)
            tmp = {system : tmp[system]['error'] for system, _ in tmp.items()}
            val = [tmp[label][Error.UNNECESSARY_INSERTION] for label in system_labels]

        if sum(val) != 0:
            # Custom labels
            label = error_type.value
            if family == Family.SYNTAX or Family.LEXICAL:
                if label == 'Error':
                    label = 'Grammar'
                # label += ' Error'

            color = color_mapping[error_type]
            if family == Family.SYNTAX:
                color = color_mapping_override[error_type]
            ax_error[plt_idx].bar(x, val, width, bottom=bottom, label=label, color=color)
        bottom = [bottom[i] + val[i] for i in range(len(val))]

    displayed_x_labels = [system_name_mapping[label] for label in system_labels]

    ax_error[plt_idx].tick_params(labelsize=font_size_x_labels)
    ax_error[plt_idx].set_xticks(np.arange(len(system_labels)))
    ax_error[plt_idx].set_xticklabels(displayed_x_labels, rotation=30, ha='right')
    ax_error[plt_idx].spines[['right', 'top']].set_visible(False)

    if family == Family.CONTENT:
        ax_error[plt_idx].set_ylabel('# Error Edits / Sent.', fontsize=font_size_y_label)

    handles, labels = ax_error[plt_idx].get_legend_handles_labels()
    ax_error[plt_idx].legend(handles[::-1], labels[::-1], loc='center left', bbox_to_anchor=legend_loc,
        fancybox=True, ncol=1, borderaxespad=1.,fontsize=font_size_legend,
        facecolor='white',edgecolor='black',framealpha=1,frameon=False,
        columnspacing=1,handlelength=1,handleheight=1,handletextpad=0.6,
        borderpad=0.2, alignment='left', 
        # title=f'{family.value} Error', title_fontproperties={'weight': 500, 'size': font_size_legend, 'style': 'italic'}
        )

    for ha in ax_error[plt_idx].legend_.legendHandles:
        ha.set_edgecolor("black")

    trans = mtrans.Affine2D().translate(10, 0)
    for t in ax_error[plt_idx].get_xticklabels():
        t.set_transform(t.get_transform()+trans)

    for text in ax_error[plt_idx].legend_.texts:
        text.set_text(fix_text(text.get_text(), syntax_error=(family == Family.SYNTAX)))

    # Set the margins a little higher than the max value
    step_dict = {
        Family.CONTENT: 0.2,
        Family.SYNTAX: 0.05,
        Family.LEXICAL: 0.1,
    }
    max_error = max([sum(x.values()) for x in error_data.values()])
    tick_range_error = np.arange(0, max_error*1.2, step=step_dict[family])
    ax_error[plt_idx].set_yticks(tick_range_error)

    ax_error[0].set_title('Conceptual Error', fontsize=font_size_title)
    ax_error[1].set_title('Syntax Error', fontsize=font_size_title)
    ax_error[2].set_title('Lexical Error', fontsize=font_size_title)

plt.tight_layout()
if savefig:
    out_filename = f'../paper/plot/error-edits.pdf'
    plt.savefig(out_filename, format="pdf", bbox_inches='tight', pad_inches=0.0)
    plt.close(fig)
else:
    plt.show()